## 1.制造数据集

In [2]:
# 生成 y = Xw+b+噪声
import torch
# w: 是一个向量（权重），在这个例子中它是一个一维张量。
# b: 是一个标量（偏置项）。
# num_examples: 表示要生成的数据点数量。
def synthetic_data(w,b,num_examples):
    # 这行代码生成了一个形状为 (num_examples, len(w)) 的张量，其中每个元素都是从均值为0，标准差为1的正态分布中随机抽取的。
    x = torch.normal(0,1,(num_examples,len(w))) # 
    # 这里使用了矩阵乘法 (torch.matmul) 来计算特征 x 和权重 w 的乘积，然后加上偏置项 b。这个操作遵循线性回归模型的公式 y = Xw + b。
    y = torch.matmul(x,w) + b # 
    # 为了使数据更加真实，这里在 y 中添加了随机噪声。噪声是从均值为0，标准差为0.01的正态分布中抽取的，与 y 的形状相同，这样每个数据点都会添加一些随机误差。
    y+=torch.normal(0,0.01,y.shape)
    return x,y.reshape((-1,1))

# 参数w
true_w = torch.tensor([2,-4.3])
# 参数b
true_b = 4.2
features,labels = synthetic_data(true_w, true_b, 1000)
print(features)
print(labels)


tensor([[-0.4687,  0.4436],
        [-0.1783, -0.3278],
        [ 0.0821,  0.3189],
        ...,
        [-0.7987, -0.6174],
        [-1.8468, -0.8209],
        [ 0.6202,  0.7983]])
tensor([[  1.3518],
        [  5.2410],
        [  2.9922],
        [ 12.4338],
        [ -3.5905],
        [  7.0147],
        [  4.3139],
        [ -6.7448],
        [  5.8094],
        [  0.8710],
        [  8.0654],
        [  8.0685],
        [  9.1178],
        [ -0.7252],
        [  5.4549],
        [ 12.4610],
        [  2.1171],
        [  2.7498],
        [  3.6371],
        [ -3.3219],
        [  5.4971],
        [  7.3217],
        [  4.1896],
        [  6.1542],
        [  7.4525],
        [  6.8589],
        [ -8.6033],
        [  7.8766],
        [  1.6972],
        [  2.6176],
        [  5.9201],
        [ -4.0090],
        [  6.6088],
        [  9.0722],
        [ 10.6596],
        [ 10.0040],
        [  0.4639],
        [  4.4285],
        [  7.9409],
        [  0.4257],
        [  9.7369]

## 2.线性回归的简单实现
### 2.1数据创建

In [3]:
import numpy as np
import torch
from torch.utils import data

# 获取数据
true_w = torch.tensor([2,-4.3])
true_b = 4.2
features,labels = synthetic_data(true_w, true_b, 1000)
print(features)

tensor([[ 1.3486,  0.0423],
        [ 1.6701, -0.9337],
        [-1.8412,  0.5899],
        ...,
        [ 0.8490,  0.0764],
        [ 0.9429, -1.5041],
        [ 0.9924,  1.4319]])


### 2.2调用框架中现有API来读取数据

In [4]:
def load_array(data_arrays,batch_size,is_train=True):
    """构造一个pytorch数据迭代器"""
    # 这里使用 TensorDataset 类来将一组数据数组打包成一个数据集。*data_arrays 是一个 unpack 操作符，它会将传入的元组或列表解包为多个参数，所以如果 data_arrays 是一个包含两个张量的元组，比如 (features, labels)，那么这行代码相当于 dataset = data.TensorDataset(features, labels)。
    dataset=data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

batch_size=10
data_iter = load_array((features,labels),batch_size)
# iter() 函数用于获取一个对象的迭代器。
# next() 函数用于从迭代器中获取下一个元素。
# 这里仅仅用于测试
next(iter(data_iter))

[tensor([[-0.0320,  1.1825],
         [ 0.3671, -0.7255],
         [ 1.5587, -0.7010],
         [-2.1174, -0.0758],
         [-0.8712,  0.4517],
         [-0.0693, -1.5572],
         [-0.3874,  1.0581],
         [ 1.6701, -0.9337],
         [ 0.7470, -1.8715],
         [-0.3038, -0.3299]]),
 tensor([[-0.9541],
         [ 8.0555],
         [10.3190],
         [ 0.2933],
         [ 0.5091],
         [10.7705],
         [-1.1157],
         [11.5572],
         [13.7489],
         [ 5.0094]])]

### 2.3使用框架预先定义好的层

In [15]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))

### 2.4初始化模型参数

In [18]:
# [0]是第0层
net[0].weight.data.normal_(0,0.1)
net[0].bias.data.fill_(0)

tensor([0.])

### 2.5计算均方误差

In [19]:
loss = nn.MSELoss()

### 2.6SGD算法

In [21]:
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

### 2.7训练过程

In [32]:
epochs = 3
for epoch in range(epochs):
    for x,y in data_iter:
        l = loss(net(x),y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    # 在每轮训练结束后，再次计算模型在完整数据集上的损失，并打印出来，以便监控训练进度和模型性能。
    l = loss(net(features),labels)
    print(f'epoch:{epoch+1} loss:{l}')
        

epoch:1 loss:0.00010324995673727244
epoch:2 loss:0.00010380881576566026
epoch:3 loss:0.0001035891255014576
